# Lemon Toy Problem A1

## Problem Statement
 
A citrus fruit juice producer in wishes to transport 1 tonne of lemons from Spain to their UK operation. They would like to choose a freight transport option that minimises the environmental impact on climate change. The freight transport options are by road in a light commercial vehicle, diesel train or sea tanker. 

Assumptions
* All data can be sourced from openLCA
* Each mode of transport leads to the same distance travelled.
* The cost of the lemons and their transport cost can be ignored.

## Specification

In this section we translate the problem statement into our abstract specification v3.

We choose the system openLCA database EcoInvent to source data for the problem and we fix the environmental impact category as $kpi =\text{"climate change - GWP100, ReCiPe Midpoint (H) V1.13"}$. 

There is no time dependence so we can drop the subscript $t$ and there is no temporary storage facility so $F_s=\emptyset$ and $S_{f_s}=0$. 

The task is indexed by $k=\{\text{'lemon supply'}\}$ which can be dropped. There is one material product flow with index in $f_m=\{\text{'lemon'}\}$. The process index set for the problem is $P=\{\text{'lemons grown in Spain'}, \text{'transport by road from Spain'}, \text{'transport by rail from Spain'}, \text{'transport by tanker from Spain'}\}$. The transport index set is $F_t=\{\text{'road'},\text{'diesel rail'},\text{'tanker'}\}$. There is a single demand so the subscript $d$ can be dropped. The total demand for lemons is $D^{total}=1$tonne.

In this problem, we minimise the environmental impact of sourcing lemons from Spain using different modes of transport. The objective is

$$
\min_{Flow_{f_m}, T_{f_t}, f_t}Flow_{f_m} EI_{kpi,f_m,p} + \sum_{f_t}T_{f_t}EI_{kpi, f_t, p}.
$$

where $Flow_{f_m}$ is the material flow of lemons in kg, $T_{f_t}$ is the quantity of lemons times distance in km and $f_t$ is the mode of transport.

The flow of lemons satisfies

$$
Flow_{f_m}=\sum_{f_t}f_{f_m,f_t},
$$

where $f_{f_m, f_t}$ is the total quantity of lemons in kg transported through transport mode $f_t$.

The quantity of lemons transported through mode $f_t$ times the distance travelled is

$$
T_{f_t} = f_{f_t}d_{p, f_m},
$$

where $d_{p, f_m}$ is the distance in km travelled by lemons using process $p$ and London. We assume the distance travelled is the same for each transport mode. Thus

$$
d_{p,f_m}=\text{Haversine}(Y, X, Y_{p, f_m}^I, X_{p, f_m}^I)
$$

where $(X,Y)$ is the Longitude and Latitude of the UK, and $(X^I_{p,f_m}, Y^I_{p,f_m})$ is the longitude and latitude of Spain from openLCA location data.

The environmental impact of fruit growth is

$$
EI_{kpi, f_m, p} = \sum_e Ef_{kpi, e}EF_{e, f_m, p} 
$$

where the set of environmental flows are indexed by $E$ and $p=$'Lemons grown in Spain'. This data can be sourced from openLCA.

The environmental impact of transporting lemons using mode $f_t$ is

$$
EI_{kpi, f_t, p} = \sum_e Ef_{kpi, e}EF_{e, f_t, p},
$$

where $p\in \{\text{'transport by road from Spain'}, \text{'transport by rail from Spain'}, \text{'transport by tanker from Spain'}\}$ using openLCA data.

We also have the constraint that the flow must be sufficient to satisfy the demand $D^{total}=1$ tonne of lemons.

$$
Flow_{f_m} \ge D^{total} C_{f_m}.
$$

where $C_{f_m}=1000$ kg per tonne.

Each decision variable satisfies $f_t \geq 0$.

## Analytical Solution

We can ignore the impact of lemon growth in the objective and distance between London and Spain in order to find the optimal solution. The objective is

$$
\min_{f_{f_t}}\sum_{f_t}f_{f_t}EI_{kpi, p, f_t},
$$ 

where 

$$
\sum_{f_t} f_{f_t} \ge 1000 \,\text{kg},\quad f_t\ge 0.
$$

So the solution is $f_{f_t}=1000$ kg for whichever tranport mode $f_t$ has smallest environmental impact $EI_{kpi, p, f_t}$ (assuming there is only one).

# Pyomo implementation

In this implementation we use mock openLCA data.

## Import modules

In [1]:
from pyomo.environ import * 
from pyomo.environ import units as u
import pandas as pd

In [2]:
model_A = ConcreteModel()
model_A.pprint()

0 Declarations: 


## Define sets

We fix the units on the material flow as kg, the quantity of lemons transported as kg. We suppose the data in the openLCA database is just sufficent to solve the problem. There are two representative elementary flows and one impact category.

In [3]:
model_A.AF = Set(initialize=['Lemons', 'Road', 'Rail', 'Tanker'], doc='All flows')
model_A.Fm = Set(initialize=['Lemons'], doc='Material flows to optimise')
model_A.Ft = Set(initialize=['Road', 'Rail', 'Tanker'], doc='Transport flows to optimise')
model_A.P = Set(initialize=['Growth of Lemons in Spain', 'Transport by road from Spain', 'Transport by rail from Spain', 'Transport by tanker from Spain'], doc='Processes in the optimisation problem')
model_A.Pm = Set(initialize=['Growth of Lemons in Spain'], doc='Processes with material output flows')
model_A.Pt = Set(initialize=['Transport by road from Spain', 'Transport by rail from Spain', 'Transport by tanker from Spain'], doc='Processes with transport output flows')
model_A.E = Set(initialize=['Elementary Flow 1', 'Elementary Flow 2'], doc='Elementary Flows from OpenLCA database')
model_A.kpi = Set(initialize=['KPI 1'], doc='KPI for optimisation') 

## Define parameters.

### Total Demand

Set the total demand of juice and the conversion factor.

In [4]:
model_A.C = Param(model_A.Fm, initialize={'Lemons': 1000}, doc='Conversion factors')
model_A.D_total = 1

### Get impact factors.

The breakdown of the KPI is broken down with dummy data into elementary flows.

In [5]:
Ef_dict = {'KPI 1': pd.Series(range(len(model_A.E)+1, 1, -1), index = model_A.E)}
model_A.Ef = Param(model_A.kpi, initialize=Ef_dict, within=Any, doc='Impact category for each elementary flow')
model_A.Ef.pprint()

Ef : Impact category for each elementary flow
    Size=1, Index=kpi, Domain=Any, Default=None, Mutable=False
    Key   : Value
    KPI 1 : E
    Elementary Flow 1    3
    Elementary Flow 2    2
    dtype: int64


### Get environmental flow parameters.

We need an elementary breakdown for each flow in $F_m$ and $F_t$. The road transport flow has been scaled so it has a lower overall impact than the others.

In [6]:
v = {k: (k != 'Transport by road from Spain') + 1 for v, k in enumerate(model_A.Pt)}
EF_Pm_dict = {f: pd.Series(range(1, len(model_A.E)+1), index = model_A.E) for f in model_A.Pm}
EF_Pt_dict = {f: v[f] * pd.Series(range(1, len(model_A.E)+1), index = model_A.E) for f in model_A.Pt}
model_A.EF_Pm = Param(model_A.Pm, initialize=EF_Pm_dict, within=Any, doc='Breakdown of material flow from process into elementary flows')
model_A.EF_Pt = Param(model_A.Pt, initialize=EF_Pt_dict, within=Any, doc='Breakdown of transport flow from process into elementary flows')
model_A.EF_Pm.pprint()
model_A.EF_Pt.pprint()

EF_Pm : Breakdown of material flow from process into elementary flows
    Size=1, Index=Pm, Domain=Any, Default=None, Mutable=False
    Key                       : Value
    Growth of Lemons in Spain : E
    Elementary Flow 1    1
    Elementary Flow 2    2
    dtype: int64
EF_Pt : Breakdown of transport flow from process into elementary flows
    Size=3, Index=Pt, Domain=Any, Default=None, Mutable=False
    Key                            : Value
      Transport by rail from Spain : E
    Elementary Flow 1    2
    Elementary Flow 2    4
    dtype: int64
      Transport by road from Spain : E
    Elementary Flow 1    1
    Elementary Flow 2    2
    dtype: int64
    Transport by tanker from Spain : E
    Elementary Flow 1    2
    Elementary Flow 2    4
    dtype: int64


Next we calulate the environmental impact of each product flow and its associated process. We need to separate out these impacts
into materal and transport because they are multiplied by a different decision variable.

In [7]:
I_Pm_dict = {p: sum(model_A.Ef['KPI 1'] * model_A.EF_Pm[p]) for p in model_A.Pm}
model_A.EI_Pm = Param(model_A.Pm, initialize=I_Pm_dict, doc='Environment impact of material flow from process')
I_Pt_dict = {p: sum(model_A.Ef['KPI 1'] * model_A.EF_Pt[p]) for p in model_A.Pt}
model_A.EI_Pt = Param(model_A.Pt, initialize=I_Pt_dict, doc='Environment impact of transport flow from process')
model_A.EI_Pm.pprint()
model_A.EI_Pt.pprint()

EI_Pm : Environment impact of material flow from process
    Size=1, Index=Pm, Domain=Any, Default=None, Mutable=False
    Key                       : Value
    Growth of Lemons in Spain :     7
EI_Pt : Environment impact of transport flow from process
    Size=3, Index=Pt, Domain=Any, Default=None, Mutable=False
    Key                            : Value
      Transport by rail from Spain :    14
      Transport by road from Spain :     7
    Transport by tanker from Spain :    14


## Define continuous decision variables

The decision variables are the material, transport service and transport flows.

In [8]:
model_A.Flow = Var(model_A.Fm, within=NonNegativeReals, doc='Material flow')
model_A.T = Var(model_A.Ft, within=NonNegativeReals, doc='Transport Service flow')
model_A.f = Var(model_A.Fm, model_A.Ft, within=NonNegativeReals, doc='Transport flow')
model_A.Flow.pprint()
model_A.T.pprint()
model_A.f.pprint()

Flow : Material flow
    Size=1, Index=Fm
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    Lemons :     0 :  None :  None : False :  True : NonNegativeReals
T : Transport Service flow
    Size=3, Index=Ft
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
      Rail :     0 :  None :  None : False :  True : NonNegativeReals
      Road :     0 :  None :  None : False :  True : NonNegativeReals
    Tanker :     0 :  None :  None : False :  True : NonNegativeReals
f : Transport flow
    Size=3, Index=f_index
    Key                  : Lower : Value : Upper : Fixed : Stale : Domain
      ('Lemons', 'Rail') :     0 :  None :  None : False :  True : NonNegativeReals
      ('Lemons', 'Road') :     0 :  None :  None : False :  True : NonNegativeReals
    ('Lemons', 'Tanker') :     0 :  None :  None : False :  True : NonNegativeReals


## Define constraints

We suppose the distance between where lemons are grown in Spain and the UK is 1000km.

For each process in $P$ we need to identify the corresponding product flow in $F$.

In [9]:
process = {'Lemons': 'Growth of Lemons in Spain', 
          'Road': 'Transport by road from Spain', 
          'Rail': 'Transport by rail from Spain', 
          'Tanker': 'Transport by tanker from Spain'}

In [10]:
model_A.d = Param(model_A.Pm, initialize = {'Growth of Lemons in Spain': 1000})
def flow_demand_rule(model, f):
    return model.Flow[f] >= model.C[f]*model.D_total
model_A.demand_constraint = Constraint(model_A.Fm, rule=flow_demand_rule)
def transport_flow_rule(model, ft):
    return model.T[ft] == sum(model.f[fm,ft]*model.d[process[fm]] for fm in model_A.Fm)
model_A.transport_constraint = Constraint(model_A.Ft, expr=transport_flow_rule)
def flow_rule(model, fm):
    return model.Flow[fm] == sum(model.f[fm,ft] for ft in model_A.Ft)
model_A.flow_constraint = Constraint(model_A.Fm, expr=flow_rule)
model_A.demand_constraint.pprint()
model_A.transport_constraint.pprint()
model_A.flow_constraint.pprint()

demand_constraint : Size=1, Index=Fm, Active=True
    Key    : Lower  : Body         : Upper : Active
    Lemons : 1000.0 : Flow[Lemons] :  +Inf :   True
transport_constraint : Size=3, Index=Ft, Active=True
    Key    : Lower : Body                              : Upper : Active
      Rail :   0.0 :     T[Rail] - 1000*f[Lemons,Rail] :   0.0 :   True
      Road :   0.0 :     T[Road] - 1000*f[Lemons,Road] :   0.0 :   True
    Tanker :   0.0 : T[Tanker] - 1000*f[Lemons,Tanker] :   0.0 :   True
flow_constraint : Size=1, Index=Fm, Active=True
    Key    : Lower : Body                                                                : Upper : Active
    Lemons :   0.0 : Flow[Lemons] - (f[Lemons,Road] + f[Lemons,Rail] + f[Lemons,Tanker]) :   0.0 :   True


## Define objective

In [11]:
model_A.obj = Objective(expr=sum(model_A.Flow[f]*model_A.EI_Pm[process[f]] for f in model_A.Fm) +
                        sum(model_A.T[f]*model_A.EI_Pt[process[f]] for f in model_A.Ft), sense=minimize)
model_A.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 7*T[Road] + 14*T[Rail] + 14*T[Tanker] + 7*Flow[Lemons]


## Apply solver

In [12]:
opt = SolverFactory("glpk")
results = opt.solve(model_A)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7007000.0
  Upper bound: 7007000.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 8
  Number of nonzeros: 12
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01540231704711914
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

## Process results

In [13]:
model_A.Flow.display()
model_A.T.display()
model_A.f.display()

Flow : Material flow
    Size=1, Index=Fm
    Key    : Lower : Value  : Upper : Fixed : Stale : Domain
    Lemons :     0 : 1000.0 :  None : False : False : NonNegativeReals
T : Transport Service flow
    Size=3, Index=Ft
    Key    : Lower : Value     : Upper : Fixed : Stale : Domain
      Rail :     0 :       0.0 :  None : False : False : NonNegativeReals
      Road :     0 : 1000000.0 :  None : False : False : NonNegativeReals
    Tanker :     0 :       0.0 :  None : False : False : NonNegativeReals
f : Transport flow
    Size=3, Index=f_index
    Key                  : Lower : Value  : Upper : Fixed : Stale : Domain
      ('Lemons', 'Rail') :     0 :    0.0 :  None : False : False : NonNegativeReals
      ('Lemons', 'Road') :     0 : 1000.0 :  None : False : False : NonNegativeReals
    ('Lemons', 'Tanker') :     0 :    0.0 :  None : False : False : NonNegativeReals


# Using openLCA data

The dataview module from the mola Python package provides simple functions to retrieve data from an openLCA database. 

## Low level function

First we load the mola modules.

In [14]:
import qgrid
import mola.dataview as dv
import mola.dataimport as di
from importlib import reload
reload(dv)

<module 'mola.dataview' from '/mnt/disk1/work/code/Python/LCA/notebooks/mola/dataview.py'>

We get a connection to a pre-defined sqlite database built from an openLCA zolca using the following function.

In [15]:
dbconn = di.get_sqlite_connection()

There are low-level functions to find the table names and then retrieve raw tables from the openlca database.

In [16]:
dv.get_table_names(dbconn)

SELECT "name" FROM "sqlite_master" WHERE "type"='table' AND "name" NOT LIKE 'sqlite_'


['TBL_ACTORS',
 'TBL_CATEGORIES',
 'TBL_CURRENCIES',
 'TBL_DQ_INDICATORS',
 'TBL_DQ_SCORES',
 'TBL_DQ_SYSTEMS',
 'TBL_EXCHANGES',
 'TBL_FLOWS',
 'TBL_FLOW_PROPERTIES',
 'TBL_FLOW_PROPERTY_FACTORS',
 'TBL_IMPACT_CATEGORIES',
 'TBL_IMPACT_FACTORS',
 'TBL_IMPACT_METHODS',
 'TBL_LOCATIONS',
 'TBL_PROCESSES',
 'TBL_PROCESS_DOCS',
 'TBL_PROCESS_LINKS',
 'TBL_PRODUCT_SYSTEMS',
 'TBL_PRODUCT_SYSTEM_PROCESSES',
 'TBL_SOURCES',
 'TBL_UNITS',
 'TBL_UNIT_GROUPS']

In [17]:
dfr = dv.get_table(dbconn, 'TBL_LOCATIONS')
dfr

SELECT * FROM "TBL_LOCATIONS"


,ID,REF_ID,NAME,DESCRIPTION,VERSION,LAST_CHANGE,F_CATEGORY,LONGITUDE,LATITUDE,CODE,KMZ
0,501,00c66f1a-036b-38f9-8b70-9cb8d925d3d9,Yemen,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1.0,1.575970e+12,None,48.350000,15.800000,YE,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
1,502,01b6e203-44b6-3835-85ed-1ddedf20d531,Tonga,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1.0,1.575970e+12,None,-175.180000,-21.200000,TO,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
2,503,0273c379-31a2-34e3-9d4a-351b4c0f4ef2,EU candidate countries 2007,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,None,0.000000,0.000000,EC-CC2007,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
3,504,0288bde0-c2d5-33f2-b576-6f61b826a650,Niue,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1.0,1.575970e+12,None,-169.860000,-19.050000,NU,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
4,505,02cb9a5a-488d-3ea0-84d9-3ae7b315e386,Western Asia,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1.0,1.575970e+12,None,43.360000,28.460000,UN-WASIA,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
...,...,...,...,...,...,...,...,...,...,...,...
573,36097873,3ac78d5e-cc6a-38b2-ad2c-18694933387a,"India, Islands",C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,0.0,1.575970e+12,None,92.880187,11.135444,IN-Islands,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
574,36097874,8088f809-4240-3a5f-b3ff-5ccad9f1e40f,Florida Reliability Coordinating Council,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,0.0,1.575970e+12,None,-81.902393,28.341743,US-FRCC,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
575,36097875,1662d8df-a168-3d22-a99f-9b180cc86f66,SERC Reliability Corporation,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,0.0,1.575970e+12,None,-86.471492,35.005265,US-SERC,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...
576,36097876,2a47e9b9-c8a9-3aa8-851b-5f388e6dfdfc,Southwest Power Pool,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,0.0,1.575970e+12,None,-97.900958,36.185030,US-SPP,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...


## High Level functions

Higher level functions provide views and joins of the raw table data in the database.

The elementary flow table maps an ID to a elementary flow name. We can obtain a table elementary flows using the following function.

In [18]:
dfr = dv.get_elementary_flows(dbconn)
qgrid.show_grid(dfr)

SELECT "ID","REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='ELEMENTARY_FLOW'


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

There are around 18000 processes in the default Ecoinvent database. So the dataview module provides a function to search for processes by name and location. The percent character is a wildcard.

In [19]:
dv.get_processes(dbconn, name=['%Lemon%'])

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE '%Lemon%'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,4578980,14a094df-5988-3a57-96eb-cca3482628b0,"market for lemon | lemon | APOS, S",LCI_RESULT,Global,4578982
1,5409534,64867712-23c4-3be5-a50e-3631e74571a6,"lemon production | lemon | APOS, S",LCI_RESULT,Spain,5409536
2,5764475,4ce4cc23-c7be-3317-a9fb-45d4d4a0a8ab,"lemon production | lemon | APOS, S",LCI_RESULT,Rest-of-World,5764477
3,12597999,9f6a92b7-80aa-3705-8cae-042e064e3090,"lemon production | lemon | APOS, S",LCI_RESULT,Turkey,12598001
4,34323860,727a873a-27de-3274-a0ae-ecd2c90b122f,"lemon production | lemon | APOS, S",LCI_RESULT,Mexico,34323862


The quantitative reference identifies an exchange that links the process to its reference flow.

In [20]:
dv.get_exchanges(dbconn, id=4578982)

SELECT "ID","F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "ID"=4578982


,ID,F_OWNER,F_FLOW,F_UNIT,RESULTING_AMOUNT_VALUE
0,4578982,4578980,132016,1056,1.0


Here F_OWNER is the process id that owns the exchange, F_FLOW is the id of the reference flow and the resulting amount is the product flow amount in the unit reference by F_UNIT. 

We can get back just the process REF_IDs from a search using

In [21]:
process_ref_ids = dv.get_process_ref_ids(dbconn, name=['Lemon%'])
process_ref_ids

SELECT "TBL_PROCESSES"."REF_ID" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'Lemon%'


['64867712-23c4-3be5-a50e-3631e74571a6',
 '4ce4cc23-c7be-3317-a9fb-45d4d4a0a8ab',
 '9f6a92b7-80aa-3705-8cae-042e064e3090',
 '727a873a-27de-3274-a0ae-ecd2c90b122f']

There is a single unit process in the default database for reference.

In [22]:
dv.get_processes(dbconn, name=['Process Z'])

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'Process Z'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,36098004,48b9f68d-cb46-4fdd-a20b-95b5df78dfdf,Process Z,UNIT_PROCESS,None,36098006


In [23]:
dv.get_product_flows(dbconn, name=['lemon%', 'orange%', 'mandarin%'])

SELECT "ID","REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW' AND ("name" LIKE 'lemon%' OR "name" LIKE 'orange%' OR "name" LIKE 'mandarin%')


,ID,REF_ID,NAME
0,129718,1f7bbd3e-fcd1-412d-8608-035b855ea735,"orange, fresh grade"
1,132016,9891d347-28f8-434d-896a-84a19a4c68a6,lemon
2,134705,ec4fd0d0-b3f5-4585-a2f9-98240429eac3,"mandarin, processing grade"
3,134735,01d3bb5a-645e-485c-9661-cdcebf93385c,"orange, processing grade"
4,135345,0b588bb3-b211-420e-80ed-f6f4fba01f8a,"mandarin, fresh grade"
5,135455,9c6b672a-661e-40c3-a9a0-902b33cc6871,mandarin


In [24]:
product_ref_ids = dv.get_product_flow_ref_ids(dbconn, name=['lemon', 'orange%p%', 'mandarin%P%'])
product_ref_ids

SELECT "REF_ID" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW' AND ("name" LIKE 'lemon' OR "name" LIKE 'orange%p%' OR "name" LIKE 'mandarin%P%')


['9891d347-28f8-434d-896a-84a19a4c68a6',
 'ec4fd0d0-b3f5-4585-a2f9-98240429eac3',
 '01d3bb5a-645e-485c-9661-cdcebf93385c']

We can get the breakdown of a product output flow from a process into elementary flows.

In [25]:
process_elementary_dfr = dv.get_process_elementary_flow(dbconn, ref_ids=process_ref_ids, limit_exchanges=None)
process_elementary_dfr

SELECT "e"."F_OWNER","e"."F_FLOW","e"."F_UNIT","e"."RESULTING_AMOUNT_VALUE","TBL_FLOWS"."FLOW_TYPE","TBL_FLOWS"."REF_ID" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (5409534,5764475,12597999,34323860)) "e" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="e"."F_FLOW" WHERE "TBL_FLOWS"."FlOW_TYPE"='ELEMENTARY_FLOW'


,64867712-23c4-3be5-a50e-3631e74571a6,4ce4cc23-c7be-3317-a9fb-45d4d4a0a8ab,9f6a92b7-80aa-3705-8cae-042e064e3090,727a873a-27de-3274-a0ae-ecd2c90b122f
REF_ID,,,,
00012c0a-9bff-4787-a7eb-56c3d2f43692,3.315358e-11,3.537200e-11,7.379112e-11,2.143797e-11
0015ec22-72cb-4af1-8c7b-0ba0d041553c,3.310572e-11,9.230810e-11,7.169612e-11,1.202275e-10
0017271e-7df5-40bc-833a-36110c1fe5d5,2.314766e-08,2.739918e-08,1.928944e-08,3.124432e-08
001790f3-fd86-4a0d-a2a1-06c7099d90c8,1.216010e-05,1.236320e-05,8.304251e-06,2.156249e-05
0017ce28-9f7a-404b-ad55-d3f43ad13cae,8.560851e-14,6.166200e-14,1.592893e-13,1.611737e-14
...,...,...,...,...
ff36578b-f403-4656-b934-81d8d4e02dc8,5.983466e-09,1.028824e-08,7.814235e-09,8.067001e-09
ff65658c-2d39-44a2-b23e-7ec3c644f064,7.704246e-06,9.875376e-06,1.338127e-05,6.799632e-06
ff6dccc1-5ebd-42c3-9fd9-3d73db7a3dd2,2.270127e-10,1.969600e-10,3.703580e-10,1.010055e-10


We cannot get a direct breakdown of a product flow into its elementary flows because of how the database
is structured. A product flow can be linked to many different processes in TBL_EXCHANGES. For example, the lemon flow is linked to processes for each region of production and there is an elementary breakdown for each of these processes. Location is part of the process not the flow.

We can see the processes that use a given product flow by using the function below.

In [26]:
product_flow_in_process_dfr = dv.get_product_flow_in_process(dbconn, product_id=[132016], limit_exchanges=None)
product_flow_in_process_dfr

SELECT "TBL_PROCESSES"."ID" "PROCESS_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","sq0"."F_FLOW" "FLOW_ID","TBL_FLOWS"."NAME" "FLOW_NAME","TBL_PROCESSES"."F_LOCATION" "PROCESS_LOCATION" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_FLOW" IN (132016)) "sq0" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW"


,PROCESS_ID,PROCESS_NAME,FLOW_ID,FLOW_NAME,PROCESS_LOCATION
0,4578980,"market for lemon | lemon | APOS, S",132016,lemon,668.0
1,5409534,"lemon production | lemon | APOS, S",132016,lemon,537.0
2,5764475,"lemon production | lemon | APOS, S",132016,lemon,955.0
3,12597999,"lemon production | lemon | APOS, S",132016,lemon,936.0
4,34323860,"lemon production | lemon | APOS, S",132016,lemon,620.0
5,36098004,Process Z,132016,lemon,NaN


We can obtain a table of impact methodologies restricted by name.

In [27]:
impact_category_dfr = dv.get_impact_categories(dbconn, method_name=['ReCiPe Midpoint (H) V1.13'])
impact_category_dfr

SELECT "TBL_IMPACT_METHODS"."ID" "methods_ID","TBL_IMPACT_METHODS"."REF_ID" "methods_REF_ID","TBL_IMPACT_METHODS"."NAME" "methods_NAME","TBL_IMPACT_CATEGORIES"."ID" "categories_ID","TBL_IMPACT_CATEGORIES"."REF_ID" "categories_REF_ID","TBL_IMPACT_CATEGORIES"."NAME" "categories_NAME" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID" WHERE "TBL_IMPACT_METHODS"."name" LIKE 'ReCiPe Midpoint (H) V1.13'


,methods_ID,methods_REF_ID,methods_NAME,categories_ID,categories_REF_ID,categories_NAME
0,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36139392,515aa8fc-27aa-3ec8-9b0b-42a8bd26a26b,metal depletion - MDP
1,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36134269,8060a79a-473c-3834-b874-6d25239d06f3,marine ecotoxicity - METPinf
2,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36135829,42b1e910-3bd2-3741-85ce-a3966798440b,climate change - GWP100
3,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36132595,dea5d939-c042-300a-916c-dd903180575f,human toxicity - HTPinf
4,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36139540,3bb7b9c4-6934-325c-974c-12d2ed74407b,fossil depletion - FDP
5,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36134054,e762c963-e5e8-33eb-829c-bff1110c5ecd,water depletion - WDP
6,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36136043,e84190cb-0e34-34a8-aca6-78de883d7409,freshwater ecotoxicity - FETPinf
7,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36139181,fa0789f2-602a-35c6-a6f3-fabc100f21e4,photochemical oxidant formation - POFP
8,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36133837,f10f3229-9d8f-30dd-8ff9-51413d46f94d,ionising radiation - IRP_HE
9,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36139478,4fa0339d-af04-336e-9772-19fdc201afeb,urban land occupation - ULOP


In [28]:
impact_category_dfr = dv.get_impact_categories(dbconn, method_name=['ReCiPe Midpoint (H) V1.13'], category_name=['Climate change - GWP100%'])
impact_category_dfr

SELECT "TBL_IMPACT_METHODS"."ID" "methods_ID","TBL_IMPACT_METHODS"."REF_ID" "methods_REF_ID","TBL_IMPACT_METHODS"."NAME" "methods_NAME","TBL_IMPACT_CATEGORIES"."ID" "categories_ID","TBL_IMPACT_CATEGORIES"."REF_ID" "categories_REF_ID","TBL_IMPACT_CATEGORIES"."NAME" "categories_NAME" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID" WHERE "TBL_IMPACT_METHODS"."name" LIKE 'ReCiPe Midpoint (H) V1.13' AND "TBL_IMPACT_CATEGORIES"."name" LIKE 'Climate change - GWP100%'


,methods_ID,methods_REF_ID,methods_NAME,categories_ID,categories_REF_ID,categories_NAME
0,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36135829,42b1e910-3bd2-3741-85ce-a3966798440b,climate change - GWP100


We need one impact category REF_ID for this calculation.

In [29]:
impact_ref_ids = dv.get_impact_category_ref_ids(dbconn, method_name=['ReCiPe Midpoint (H) V1.13'], category_name=['Climate change - GWP100%'])
impact_ref_ids

SELECT "TBL_IMPACT_METHODS"."ID" "methods_ID","TBL_IMPACT_METHODS"."REF_ID" "methods_REF_ID","TBL_IMPACT_METHODS"."NAME" "methods_NAME","TBL_IMPACT_CATEGORIES"."REF_ID" "categories_REF_ID" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID" WHERE "TBL_IMPACT_METHODS"."name" LIKE 'ReCiPe Midpoint (H) V1.13' AND "TBL_IMPACT_CATEGORIES"."name" LIKE 'Climate change - GWP100%'


['42b1e910-3bd2-3741-85ce-a3966798440b']

The following function gets the impact factors for a methodology for a given list of category reference ids. The Dataframe returned has rows indexed by elementary flow reference id. The column names are reference ids for the impact category in the openLCA database.

In [30]:
impact_element_dfr = dv.get_impact_category_elementary_flow(dbconn, ref_ids=impact_ref_ids, limit_factors=None)
impact_element_dfr

SELECT "ic"."REF_ID" "IMPACT_CATEGORY_REF_ID","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_IMPACT_FACTORS"."value" FROM (SELECT "ID","REF_ID" FROM "TBL_IMPACT_CATEGORIES" WHERE "REF_ID" IN ('42b1e910-3bd2-3741-85ce-a3966798440b')) "ic" LEFT JOIN "TBL_IMPACT_FACTORS" ON "TBL_IMPACT_FACTORS"."F_IMPACT_CATEGORY"="ic"."ID" LEFT JOIN "TBL_FLOWS" ON "TBL_IMPACT_FACTORS"."F_FLOW"="TBL_FLOWS"."ID"


IMPACT_CATEGORY_REF_ID,42b1e910-3bd2-3741-85ce-a3966798440b
FLOW_REF_ID,
033d3a16-e1a2-4ce8-8769-f4ea867801ca,725.0
04a6edb8-624a-484e-8f85-d771657adae7,151.0
050da0cd-2957-45a9-ae7a-ace372083fe5,77.0
06a4fedf-dab4-4ef1-90df-1d0ce8fe6477,1890.0
0795345f-c7ae-410c-ad25-1845784c75f5,25.0
...,...
f8cf5fd7-9f94-4870-9e6a-768deae9b766,17200.0
f9749677-9c9f-4678-ab55-c607dfdc2cb9,1.0
fa0c2bee-8dd9-4f8a-8489-b1f3b43de958,1430.0


# Pyomo implementation using openLCA data

In [31]:
olca_model_A = ConcreteModel()

## Define sets

Set are defined using openLCA reference ids. The relevant openLCA transport flows are shown below.

In [32]:
dfr = dv.get_product_flows(dbconn, name=['transport, freight, light commercial vehicle',
                                   'transport, freight train', 
                                   'transport, freight, sea, container ship'])
flow_ref_ids = dfr['REF_ID'].to_list()
dfr

SELECT "ID","REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW' AND ("name" LIKE 'transport, freight, light commercial vehicle' OR "name" LIKE 'transport, freight train' OR "name" LIKE 'transport, freight, sea, container ship')


,ID,REF_ID,NAME
0,130167,0ace02fa-eca5-482d-a829-c18e46a52db4,"transport, freight train"
1,131603,090f56b8-6a83-48c1-ae26-234d04771e53,"transport, freight, light commercial vehicle"
2,135867,2741cea8-327f-4e0f-9401-b10858dc68f8,"transport, freight, sea, container ship"


The relevant processes for the problem are shown below. 

In [33]:
production_dfr = dv.get_processes(dbconn, name=['lemon production%'], location=['Spain'])
production_dfr

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'lemon production%' AND "TBL_LOCATIONS"."NAME" LIKE 'Spain'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,5409534,64867712-23c4-3be5-a50e-3631e74571a6,"lemon production | lemon | APOS, S",LCI_RESULT,Spain,5409536


In [34]:
transport1_dfr = dv.get_processes(dbconn, name=[
    'transport, freight, light commercial vehicle | transport, freight, light commercial vehicle | APOS, S',
    'transport, freight train, diesel | transport, freight train | APOS, S'], location=['Europe without Switzerland'])
transport2_dfr = dv.get_processes(dbconn, name=[
    'transport, freight, sea, container ship | transport, freight, sea, container ship | APOS, S'], location=['Global'])
transport_ref_ids = transport1_dfr['REF_ID'].to_list() + transport2_dfr['REF_ID'].to_list()
material_ref_ids = production_dfr['REF_ID'].to_list()
process_ref_ids = material_ref_ids + transport_ref_ids
transport1_dfr.append(transport2_dfr)

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE ("TBL_PROCESSES"."NAME" LIKE 'transport, freight, light commercial vehicle | transport, freight, light commercial vehicle | APOS, S' OR "TBL_PROCESSES"."NAME" LIKE 'transport, freight train, diesel | transport, freight train | APOS, S') AND "TBL_LOCATIONS"."NAME" LIKE 'Europe without Switzerland'
SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'transport, freight, sea, container ship | transport, freight, sea, 

,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,10990570,b0d5d6ec-77f4-3213-b613-2391089d5f96,"transport, freight train, diesel | transport, ...",LCI_RESULT,Europe without Switzerland,10990572
1,33642886,13e79207-71e6-36c9-be76-716700ea1e81,"transport, freight, light commercial vehicle |...",LCI_RESULT,Europe without Switzerland,33642888
0,25967156,3886c7a5-bb83-41d8-9b20-41c0df9742ce,"transport, freight, sea, container ship | tran...",LCI_RESULT,Global,25967158


In [35]:
all_flow_ids = dv.get_product_flow_ref_ids(dbconn)
olca_model_A.AF = Set(initialize=all_flow_ids, doc='All flows')
olca_model_A.Fm = Set(initialize=dv.get_product_flow_ref_ids(dbconn, name=['lemon']), doc='Material flows to optimise')
olca_model_A.Ft = Set(initialize=flow_ref_ids, doc='Transport flows to optimise')
olca_model_A.P = Set(initialize=process_ref_ids, doc='Processes in the optimisation problem')
olca_model_A.Pm = Set(initialize=material_ref_ids, doc='Processes with material output flows')
olca_model_A.Pt = Set(initialize=transport_ref_ids, doc='Processes with transport output flows')
olca_model_A.E = Set(initialize=dv.get_elementary_flow_ref_ids(dbconn), doc='Elementary Flows from OpenLCA database')
olca_model_A.P.pprint()

SELECT "REF_ID" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW'
SELECT "REF_ID" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW' AND "name" LIKE 'lemon'
SELECT "REF_ID" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='ELEMENTARY_FLOW'
P : Processes in the optimisation problem
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    4 : {'64867712-23c4-3be5-a50e-3631e74571a6', 'b0d5d6ec-77f4-3213-b613-2391089d5f96', '13e79207-71e6-36c9-be76-716700ea1e81', '3886c7a5-bb83-41d8-9b20-41c0df9742ce'}


We find an impact category reference id and use it so set the $kpi$ in the model.

In [36]:
# Set the KPI
kpi_ref_id = dv.get_impact_category_ref_ids(dbconn, method_name=['ReCiPe Midpoint (H) V1.13'], 
                                            category_name=['Climate change - GWP100%'])
olca_model_A.kpi = Set(initialize=kpi_ref_id, doc='KPI for optimisation')
olca_model_A.kpi.pprint()

SELECT "TBL_IMPACT_METHODS"."ID" "methods_ID","TBL_IMPACT_METHODS"."REF_ID" "methods_REF_ID","TBL_IMPACT_METHODS"."NAME" "methods_NAME","TBL_IMPACT_CATEGORIES"."REF_ID" "categories_REF_ID" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID" WHERE "TBL_IMPACT_METHODS"."name" LIKE 'ReCiPe Midpoint (H) V1.13' AND "TBL_IMPACT_CATEGORIES"."name" LIKE 'Climate change - GWP100%'
kpi : KPI for optimisation
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    1 : {'42b1e910-3bd2-3741-85ce-a3966798440b',}


## Define parameters

### Total demand

In [37]:
olca_model_A.C = Param(olca_model_A.Fm, initialize={'9891d347-28f8-434d-896a-84a19a4c68a6': 1000}, doc='Conversion factors')
olca_model_A.D_total = 1

### Get impact factors

We obtain the breakdown of the $kpi$ in terms of elementary flows.

In [38]:
recipe_climate_impact_factors = dv.get_impact_category_elementary_flow(dbconn, ref_ids=kpi_ref_id)
olca_model_A.Ef = Param(olca_model_A.E, olca_model_A.kpi, initialize=recipe_climate_impact_factors.stack().to_dict(),
                        within=Any, doc='Impact category for each elementary flow', default = 0)
olca_model_A.Ef.pprint()

SELECT "ic"."REF_ID" "IMPACT_CATEGORY_REF_ID","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_IMPACT_FACTORS"."value" FROM (SELECT "ID","REF_ID" FROM "TBL_IMPACT_CATEGORIES" WHERE "REF_ID" IN ('42b1e910-3bd2-3741-85ce-a3966798440b')) "ic" LEFT JOIN "TBL_IMPACT_FACTORS" ON "TBL_IMPACT_FACTORS"."F_IMPACT_CATEGORY"="ic"."ID" LEFT JOIN "TBL_FLOWS" ON "TBL_IMPACT_FACTORS"."F_FLOW"="TBL_FLOWS"."ID"
Ef : Impact category for each elementary flow
    Size=3930, Index=Ef_index, Domain=Any, Default=0, Mutable=False
    Key                                                                              : Value
    ('033d3a16-e1a2-4ce8-8769-f4ea867801ca', '42b1e910-3bd2-3741-85ce-a3966798440b') :   725.0
    ('04a6edb8-624a-484e-8f85-d771657adae7', '42b1e910-3bd2-3741-85ce-a3966798440b') :   151.0
    ('050da0cd-2957-45a9-ae7a-ace372083fe5', '42b1e910-3bd2-3741-85ce-a3966798440b') :    77.0
    ('06a4fedf-dab4-4ef1-90df-1d0ce8fe6477', '42b1e910-3bd2-3741-85ce-a3966798440b') :  1890.0
    ('0795345f-c7ae-410c-

### Get environmental flow parameters.

We also need an elementary flow breakdown for each product flow in $F_m$ and $F_t$.

In [39]:
material_process_elementary_dfr = dv.get_process_elementary_flow(dbconn, ref_ids=production_dfr['REF_ID'].to_list(), limit_exchanges=None)
material_process_elementary_dfr

SELECT "e"."F_OWNER","e"."F_FLOW","e"."F_UNIT","e"."RESULTING_AMOUNT_VALUE","TBL_FLOWS"."FLOW_TYPE","TBL_FLOWS"."REF_ID" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (5409534)) "e" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="e"."F_FLOW" WHERE "TBL_FLOWS"."FlOW_TYPE"='ELEMENTARY_FLOW'


,64867712-23c4-3be5-a50e-3631e74571a6
REF_ID,
00012c0a-9bff-4787-a7eb-56c3d2f43692,3.315358e-11
0015ec22-72cb-4af1-8c7b-0ba0d041553c,3.310572e-11
0017271e-7df5-40bc-833a-36110c1fe5d5,2.314766e-08
001790f3-fd86-4a0d-a2a1-06c7099d90c8,1.216010e-05
0017ce28-9f7a-404b-ad55-d3f43ad13cae,8.560851e-14
...,...
ff36578b-f403-4656-b934-81d8d4e02dc8,5.983466e-09
ff65658c-2d39-44a2-b23e-7ec3c644f064,7.704246e-06
ff6dccc1-5ebd-42c3-9fd9-3d73db7a3dd2,2.270127e-10


In [40]:
transport_process_elementary_dfr = dv.get_process_elementary_flow(dbconn, ref_ids=transport_ref_ids, limit_exchanges=None)
transport_process_elementary_dfr

SELECT "e"."F_OWNER","e"."F_FLOW","e"."F_UNIT","e"."RESULTING_AMOUNT_VALUE","TBL_FLOWS"."FLOW_TYPE","TBL_FLOWS"."REF_ID" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (10990570,25967156,33642886)) "e" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="e"."F_FLOW" WHERE "TBL_FLOWS"."FlOW_TYPE"='ELEMENTARY_FLOW'


,b0d5d6ec-77f4-3213-b613-2391089d5f96,3886c7a5-bb83-41d8-9b20-41c0df9742ce,13e79207-71e6-36c9-be76-716700ea1e81
REF_ID,,,
00012c0a-9bff-4787-a7eb-56c3d2f43692,3.483849e-14,1.065112e-15,1.343178e-12
0015ec22-72cb-4af1-8c7b-0ba0d041553c,2.219304e-14,4.672549e-16,5.893830e-13
0017271e-7df5-40bc-833a-36110c1fe5d5,1.925405e-08,1.270445e-09,3.141943e-07
001790f3-fd86-4a0d-a2a1-06c7099d90c8,3.069829e-06,1.130097e-07,1.571742e-04
0017ce28-9f7a-404b-ad55-d3f43ad13cae,1.561346e-16,6.389158e-18,9.859565e-15
...,...,...,...
ff36578b-f403-4656-b934-81d8d4e02dc8,3.301364e-09,1.808864e-10,1.065688e-07
ff65658c-2d39-44a2-b23e-7ec3c644f064,1.464274e-06,6.263453e-08,6.750265e-05
ff6dccc1-5ebd-42c3-9fd9-3d73db7a3dd2,9.328170e-12,3.999789e-13,2.801591e-10


In [41]:
material_process_elementary_dfr.stack().append(
transport_process_elementary_dfr.stack())

REF_ID                                                                    
00012c0a-9bff-4787-a7eb-56c3d2f43692  64867712-23c4-3be5-a50e-3631e74571a6    3.315358e-11
0015ec22-72cb-4af1-8c7b-0ba0d041553c  64867712-23c4-3be5-a50e-3631e74571a6    3.310572e-11
0017271e-7df5-40bc-833a-36110c1fe5d5  64867712-23c4-3be5-a50e-3631e74571a6    2.314766e-08
001790f3-fd86-4a0d-a2a1-06c7099d90c8  64867712-23c4-3be5-a50e-3631e74571a6    1.216010e-05
0017ce28-9f7a-404b-ad55-d3f43ad13cae  64867712-23c4-3be5-a50e-3631e74571a6    8.560851e-14
                                                                                  ...     
ffaaffd3-5deb-4508-9e5f-e47f551ac2b8  3886c7a5-bb83-41d8-9b20-41c0df9742ce    7.680476e-15
                                      13e79207-71e6-36c9-be76-716700ea1e81    7.796265e-13
ffbe8447-6d78-46dd-8378-47019a82ce82  b0d5d6ec-77f4-3213-b613-2391089d5f96    5.815804e-11
                                      3886c7a5-bb83-41d8-9b20-41c0df9742ce    2.034394e-12
               

In [42]:
olca_model_A.EF_Pm = Param(olca_model_A.E, olca_model_A.Pm, initialize=material_process_elementary_dfr.stack().to_dict(),
                        within=Any, doc='Breakdown of material flows into elementary flows', default = 0)
olca_model_A.EF_Pt = Param(olca_model_A.E, olca_model_A.Pt, initialize=transport_process_elementary_dfr.stack().to_dict(),
                        within=Any, doc='Breakdown of transport flows into elementary flows', default = 0)
olca_model_A.EF_Pm.pprint()
olca_model_A.EF_Pt.pprint()

EF_Pm : Breakdown of material flows into elementary flows
    Size=3930, Index=EF_Pm_index, Domain=Any, Default=0, Mutable=False
    Key                                                                              : Value
    ('00012c0a-9bff-4787-a7eb-56c3d2f43692', '64867712-23c4-3be5-a50e-3631e74571a6') :           3.3153576e-11
    ('0015ec22-72cb-4af1-8c7b-0ba0d041553c', '64867712-23c4-3be5-a50e-3631e74571a6') :           3.3105721e-11
    ('0017271e-7df5-40bc-833a-36110c1fe5d5', '64867712-23c4-3be5-a50e-3631e74571a6') :  2.3147662999999995e-08
    ('001790f3-fd86-4a0d-a2a1-06c7099d90c8', '64867712-23c4-3be5-a50e-3631e74571a6') :  1.2160102999999999e-05
    ('0017ce28-9f7a-404b-ad55-d3f43ad13cae', '64867712-23c4-3be5-a50e-3631e74571a6') :           8.5608507e-14
    ('006aa3f7-59ba-450f-aa45-a2b2d1752647', '64867712-23c4-3be5-a50e-3631e74571a6') :            2.330763e-09
    ('00907a61-b501-4f47-b688-1dc2b51d48c1', '64867712-23c4-3be5-a50e-3631e74571a6') :  1.9268413999999999e-10
 

    ('5c9d8310-f7e1-4e98-8e73-7eece0435676', '64867712-23c4-3be5-a50e-3631e74571a6') :            1.745604e-09
    ('5ce378a0-b48d-471c-977d-79681521efde', '64867712-23c4-3be5-a50e-3631e74571a6') :   7.112895599999999e-08
    ('5ce634de-b810-4533-9efb-719c38fe7c09', '64867712-23c4-3be5-a50e-3631e74571a6') :           0.00016776126
    ('5cf4e418-9cd0-5139-bfc2-5d8a8ffda5ff', '64867712-23c4-3be5-a50e-3631e74571a6') :           3.5353102e-14
    ('5cf55e96-d9dc-4b84-9c7e-a4b8ade70bf4', '64867712-23c4-3be5-a50e-3631e74571a6') :           5.9229744e-08
    ('5cf8884f-62a7-4cc0-8d5a-efe89beac559', '64867712-23c4-3be5-a50e-3631e74571a6') :           3.2491793e-16
    ('5d0058e0-ad1b-4c15-b0ff-a665bb04154f', '64867712-23c4-3be5-a50e-3631e74571a6') :           6.3538662e-15
    ('5d02269c-1130-47a5-aa86-00d79802dc88', '64867712-23c4-3be5-a50e-3631e74571a6') :           1.8053374e-12
    ('5d368100-b1bc-4456-8420-e469edccf349', '64867712-23c4-3be5-a50e-3631e74571a6') :           4.7959088e-05
 

    ('d1679eda-eac2-4be4-876f-3be480c53dc4', '64867712-23c4-3be5-a50e-3631e74571a6') :           1.5936988e-05
    ('d16c8806-7701-4fbb-b68d-9b2d5d083d7a', '64867712-23c4-3be5-a50e-3631e74571a6') :           1.0180409e-05
    ('d17f1c53-ff95-4005-9f1b-131ca9791cdb', '64867712-23c4-3be5-a50e-3631e74571a6') :           3.5088041e-09
    ('d1812c3b-7f5e-4ce3-80b2-e599642168cf', '64867712-23c4-3be5-a50e-3631e74571a6') :  1.2828402000000001e-11
    ('d18838bc-54cd-5d6f-a8f3-9b94cfc8472d', '64867712-23c4-3be5-a50e-3631e74571a6') :           2.5769676e-10
    ('d1a0b5a7-beb7-47e6-83b7-99560324008f', '64867712-23c4-3be5-a50e-3631e74571a6') :  4.5155618000000005e-13
    ('d1c0567e-ff1a-4cb7-a4f0-de28b2d99b63', '64867712-23c4-3be5-a50e-3631e74571a6') :  3.0459578999999995e-06
    ('d1fd0de4-afa8-4385-b91b-023493b68782', '64867712-23c4-3be5-a50e-3631e74571a6') :   9.219137599999998e-08
    ('d2025041-20ee-41bb-b2c8-47631e839ff0', '64867712-23c4-3be5-a50e-3631e74571a6') :           2.5013466e-11
 

    Key                                                                              : Value
    ('00012c0a-9bff-4787-a7eb-56c3d2f43692', '13e79207-71e6-36c9-be76-716700ea1e81') :           1.3431777e-12
    ('00012c0a-9bff-4787-a7eb-56c3d2f43692', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           1.0651121e-15
    ('00012c0a-9bff-4787-a7eb-56c3d2f43692', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :            3.483849e-14
    ('0015ec22-72cb-4af1-8c7b-0ba0d041553c', '13e79207-71e6-36c9-be76-716700ea1e81') :             5.89383e-13
    ('0015ec22-72cb-4af1-8c7b-0ba0d041553c', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           4.6725492e-16
    ('0015ec22-72cb-4af1-8c7b-0ba0d041553c', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :  2.2193042000000003e-14
    ('0017271e-7df5-40bc-833a-36110c1fe5d5', '13e79207-71e6-36c9-be76-716700ea1e81') :  3.1419428000000005e-07
    ('0017271e-7df5-40bc-833a-36110c1fe5d5', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           1.2704451e-09
    ('0017271e-7df5

    ('2925111b-3464-4e34-9d02-b73c1c033e91', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           2.7465579e-05
    ('295060ba-3c3d-45e4-99ca-e17b1204b553', '13e79207-71e6-36c9-be76-716700ea1e81') :   7.086995200000001e-08
    ('295060ba-3c3d-45e4-99ca-e17b1204b553', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           6.3762601e-09
    ('295060ba-3c3d-45e4-99ca-e17b1204b553', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           4.5057496e-09
    ('295c9740-6fdb-4676-9eb8-15e3786f713d', '13e79207-71e6-36c9-be76-716700ea1e81') :           0.00019309952
    ('295c9740-6fdb-4676-9eb8-15e3786f713d', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           5.6524531e-06
    ('295c9740-6fdb-4676-9eb8-15e3786f713d', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           1.0053118e-05
    ('29630a65-f38c-48a5-9744-c0121f586640', '13e79207-71e6-36c9-be76-716700ea1e81') :           0.00018722205
    ('29630a65-f38c-48a5-9744-c0121f586640', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           1.3259575e-07
 

    ('5253e322-7de5-42aa-9167-daf22ff44031', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :  1.1059290000000002e-16
    ('527d6cb5-bbd7-403d-8dcc-b6678fb123f5', '13e79207-71e6-36c9-be76-716700ea1e81') :           7.5549469e-09
    ('527d6cb5-bbd7-403d-8dcc-b6678fb123f5', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           1.8422743e-11
    ('527d6cb5-bbd7-403d-8dcc-b6678fb123f5', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           1.0961321e-09
    ('52a00fec-4289-485c-a56e-380de43e1b97', '13e79207-71e6-36c9-be76-716700ea1e81') :  7.5572632999999995e-22
    ('52a00fec-4289-485c-a56e-380de43e1b97', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :   5.697092399999999e-25
    ('52a00fec-4289-485c-a56e-380de43e1b97', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           2.4373767e-23
    ('531c5bca-0304-4c3f-a058-c60371df26fb', '13e79207-71e6-36c9-be76-716700ea1e81') :           3.8137449e-13
    ('531c5bca-0304-4c3f-a058-c60371df26fb', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :  2.4713682999999994e-16
 

    ('8c690557-25ce-48f2-91b9-58126e82aaf6', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :  2.8966283000000004e-16
    ('8c690557-25ce-48f2-91b9-58126e82aaf6', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :  1.8049461000000002e-14
    ('8c75e7ab-8ab8-41e4-b394-c166ff5b050d', '13e79207-71e6-36c9-be76-716700ea1e81') :            0.0017164124
    ('8c75e7ab-8ab8-41e4-b394-c166ff5b050d', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           1.6936246e-06
    ('8c75e7ab-8ab8-41e4-b394-c166ff5b050d', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           3.7930449e-05
    ('8c823f73-5fc6-4334-8a96-651365e7b542', '13e79207-71e6-36c9-be76-716700ea1e81') :  2.7056138999999997e-13
    ('8c823f73-5fc6-4334-8a96-651365e7b542', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :  2.6654303000000003e-15
    ('8c823f73-5fc6-4334-8a96-651365e7b542', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :   6.205344399999999e-15
    ('8c8ffaa5-84ed-4668-ba7d-80fd0f47013f', '13e79207-71e6-36c9-be76-716700ea1e81') :           1.1924208e-06
 

    ('a768b3a4-dce7-48f2-bb44-89c8a4ba00d1', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           3.2018444e-09
    ('a768b3a4-dce7-48f2-bb44-89c8a4ba00d1', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           1.6848678e-08
    ('a7831686-271e-4054-ac34-cf55814f902c', '13e79207-71e6-36c9-be76-716700ea1e81') :           7.9010525e-10
    ('a7831686-271e-4054-ac34-cf55814f902c', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :  1.8066617000000002e-13
    ('a7831686-271e-4054-ac34-cf55814f902c', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :  6.3983699999999995e-12
    ('a793710a-1990-4e65-a95e-cfbbc5332355', '13e79207-71e6-36c9-be76-716700ea1e81') :           1.8255385e-06
    ('a793710a-1990-4e65-a95e-cfbbc5332355', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           1.3735029e-11
    ('a793710a-1990-4e65-a95e-cfbbc5332355', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           1.4571165e-10
    ('a794b444-2b77-4449-8592-0efda101b0c2', '13e79207-71e6-36c9-be76-716700ea1e81') :  2.6341640000000003e-11
 

    ('e04d6f3d-1cb2-4ab1-aef9-fd0221028fe2', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :   5.660435000000001e-13
    ('e063ee9c-9850-42b5-b01e-4cc9b5ad7152', '13e79207-71e6-36c9-be76-716700ea1e81') :  0.00046847452000000006
    ('e063ee9c-9850-42b5-b01e-4cc9b5ad7152', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :            3.031205e-06
    ('e063ee9c-9850-42b5-b01e-4cc9b5ad7152', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :           1.1946981e-05
    ('e070bd24-fd16-4963-8f4d-78bb4c402400', '13e79207-71e6-36c9-be76-716700ea1e81') :            1.030659e-08
    ('e070bd24-fd16-4963-8f4d-78bb4c402400', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :            1.761352e-11
    ('e070bd24-fd16-4963-8f4d-78bb4c402400', 'b0d5d6ec-77f4-3213-b613-2391089d5f96') :   9.173823299999999e-11
    ('e0e605c8-370b-4b5a-b6d4-2e7a5a1a6709', '13e79207-71e6-36c9-be76-716700ea1e81') :  5.3308137999999994e-08
    ('e0e605c8-370b-4b5a-b6d4-2e7a5a1a6709', '3886c7a5-bb83-41d8-9b20-41c0df9742ce') :           1.4499302e-10
 

Next we calculate the environmental impact of each of these flows.

In [43]:
I_Pm_dict = {p: sum(olca_model_A.Ef[e, kpi_ref_id[0]]*olca_model_A.EF_Pm[e, p] for e in olca_model_A.E) for p in olca_model_A.Pm}
I_Pt_dict = {p: sum(olca_model_A.Ef[e, kpi_ref_id[0]]*olca_model_A.EF_Pt[e, p] for e in olca_model_A.E) for p in olca_model_A.Pt}
olca_model_A.EI_Pm = Param(olca_model_A.Pm, initialize=I_Pm_dict, doc='Environment Impact of flow from material process')
olca_model_A.EI_Pt = Param(olca_model_A.Pt, initialize=I_Pt_dict, doc='Environment Impact of flow from transport process')
olca_model_A.EI_Pm.pprint()
olca_model_A.EI_Pt.pprint()

EI_Pm : Environment Impact of flow from material process
    Size=1, Index=Pm, Domain=Any, Default=None, Mutable=False
    Key                                  : Value
    64867712-23c4-3be5-a50e-3631e74571a6 : 0.13944742151786754
EI_Pt : Environment Impact of flow from transport process
    Size=3, Index=Pt, Domain=Any, Default=None, Mutable=False
    Key                                  : Value
    13e79207-71e6-36c9-be76-716700ea1e81 :     1.84302999202496
    3886c7a5-bb83-41d8-9b20-41c0df9742ce : 0.009366999860024065
    b0d5d6ec-77f4-3213-b613-2391089d5f96 : 0.056119557120436626


### Distances

For each process in $P$ we need to identify the corresponding product flow in $F$. The following function does this.

In [44]:
process_dfr = dv.get_process_product_flow(dbconn, ref_ids=material_ref_ids + transport_ref_ids)
process = dict(zip(process_dfr['FLOW_REF_ID'], process_dfr['PROCESS_REF_ID']))
process_dfr

SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_FLOWS"."NAME" "FLOW_NAME" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (5409534,10990570,25967156,33642886)) "e" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="e"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="e"."F_OWNER" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME
0,64867712-23c4-3be5-a50e-3631e74571a6,"lemon production | lemon | APOS, S",Spain,9891d347-28f8-434d-896a-84a19a4c68a6,lemon
1,b0d5d6ec-77f4-3213-b613-2391089d5f96,"transport, freight train, diesel | transport, ...",Europe without Switzerland,0ace02fa-eca5-482d-a829-c18e46a52db4,"transport, freight train"
2,3886c7a5-bb83-41d8-9b20-41c0df9742ce,"transport, freight, sea, container ship | tran...",Global,2741cea8-327f-4e0f-9401-b10858dc68f8,"transport, freight, sea, container ship"
3,13e79207-71e6-36c9-be76-716700ea1e81,"transport, freight, light commercial vehicle |...",Europe without Switzerland,090f56b8-6a83-48c1-ae26-234d04771e53,"transport, freight, light commercial vehicle"


In [45]:
# process = {'9891d347-28f8-434d-896a-84a19a4c68a6': '64867712-23c4-3be5-a50e-3631e74571a6', 
#           '0ace02fa-eca5-482d-a829-c18e46a52db4': 'b0d5d6ec-77f4-3213-b613-2391089d5f96', 
#           '090f56b8-6a83-48c1-ae26-234d04771e53': '13e79207-71e6-36c9-be76-716700ea1e81', 
#           '2741cea8-327f-4e0f-9401-b10858dc68f8': '3886c7a5-bb83-41d8-9b20-41c0df9742ce'}

We calculate the distance between the UK and Spain using openLCA data. We only want the lemon production location because that's where the lemon originate. Positive latitude is N, positive longitude is E.

In [46]:
locations = dv.get_process_locations(dbconn, list(process.values()))
locations

SELECT "TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_LOCATIONS"."LATITUDE","TBL_LOCATIONS"."LONGITUDE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."REF_ID" IN ('64867712-23c4-3be5-a50e-3631e74571a6','b0d5d6ec-77f4-3213-b613-2391089d5f96','3886c7a5-bb83-41d8-9b20-41c0df9742ce','13e79207-71e6-36c9-be76-716700ea1e81')


,REF_ID,NAME,LATITUDE,LONGITUDE
0,64867712-23c4-3be5-a50e-3631e74571a6,"lemon production | lemon | APOS, S",40.220000,-3.640000
1,b0d5d6ec-77f4-3213-b613-2391089d5f96,"transport, freight train, diesel | transport, ...",48.597508,6.421103
2,3886c7a5-bb83-41d8-9b20-41c0df9742ce,"transport, freight, sea, container ship | tran...",0.000000,0.000000
3,13e79207-71e6-36c9-be76-716700ea1e81,"transport, freight, light commercial vehicle |...",48.597508,6.421103


We also need the location of the UK which is part of the Task. Currently, this is hardcoded. We use a package
to calculate the distance in km between latitudes and longitudes using a Haversine formula.

In [47]:
import pygeodesy.formy as pygeo 
# London, UK
uk_latitude = 51.5074 # N
uk_longitude = -0.1278 # E

def dist(row):
    return pygeo.haversine(uk_latitude, uk_longitude, row['LATITUDE'], row['LONGITUDE']) / 1000
locations['dist'] = locations.apply(dist, axis=1)
locations

,REF_ID,NAME,LATITUDE,LONGITUDE,dist
0,64867712-23c4-3be5-a50e-3631e74571a6,"lemon production | lemon | APOS, S",40.220000,-3.640000,1283.834155
1,b0d5d6ec-77f4-3213-b613-2391089d5f96,"transport, freight train, diesel | transport, ...",48.597508,6.421103,568.268672
2,3886c7a5-bb83-41d8-9b20-41c0df9742ce,"transport, freight, sea, container ship | tran...",0.000000,0.000000,5727.382053
3,13e79207-71e6-36c9-be76-716700ea1e81,"transport, freight, light commercial vehicle |...",48.597508,6.421103,568.268672


In [48]:
dist_init = {'64867712-23c4-3be5-a50e-3631e74571a6': 
         locations.loc[locations['REF_ID']=='64867712-23c4-3be5-a50e-3631e74571a6', 'dist'].values[0]}
olca_model_A.dist = Param(olca_model_A.Pm, within=NonNegativeReals, initialize = dist_init, doc='Distance')
olca_model_A.dist.pprint()

dist : Distance
    Size=1, Index=Pm, Domain=NonNegativeReals, Default=None, Mutable=False
    Key                                  : Value
    64867712-23c4-3be5-a50e-3631e74571a6 : 1283.834155118865


CHECK UNITS

## Define continuous decision variables

In [49]:
olca_model_A.Flow = Var(olca_model_A.Fm, within=NonNegativeReals, doc='Material flow')
olca_model_A.T = Var(olca_model_A.Ft, within=NonNegativeReals, doc='Transport Service flow')
olca_model_A.f = Var(olca_model_A.Fm, olca_model_A.Ft, within=NonNegativeReals, doc='Transport flow')
olca_model_A.Flow.pprint()
olca_model_A.T.pprint()
olca_model_A.f.pprint()

Flow : Material flow
    Size=1, Index=Fm
    Key                                  : Lower : Value : Upper : Fixed : Stale : Domain
    9891d347-28f8-434d-896a-84a19a4c68a6 :     0 :  None :  None : False :  True : NonNegativeReals
T : Transport Service flow
    Size=3, Index=Ft
    Key                                  : Lower : Value : Upper : Fixed : Stale : Domain
    090f56b8-6a83-48c1-ae26-234d04771e53 :     0 :  None :  None : False :  True : NonNegativeReals
    0ace02fa-eca5-482d-a829-c18e46a52db4 :     0 :  None :  None : False :  True : NonNegativeReals
    2741cea8-327f-4e0f-9401-b10858dc68f8 :     0 :  None :  None : False :  True : NonNegativeReals
f : Transport flow
    Size=3, Index=f_index
    Key                                                                              : Lower : Value : Upper : Fixed : Stale : Domain
    ('9891d347-28f8-434d-896a-84a19a4c68a6', '090f56b8-6a83-48c1-ae26-234d04771e53') :     0 :  None :  None : False :  True : NonNegativeReals
    ('9

## Define constraints

In [50]:
def flow_demand_rule(model, f):
    return model.Flow[f] >= model.C[f]*model.D_total
olca_model_A.demand_constraint = Constraint(olca_model_A.Fm, rule=flow_demand_rule)
def transport_flow_rule(model, ft):
    return model.T[ft] == sum(model.f[fm,ft]*model.dist[process[fm]] for fm in olca_model_A.Fm)
olca_model_A.transport_constraint = Constraint(olca_model_A.Ft, expr=transport_flow_rule)
def flow_rule(model, fm):
    return model.Flow[fm] == sum(model.f[fm,ft] for ft in olca_model_A.Ft)
olca_model_A.flow_constraint = Constraint(olca_model_A.Fm, expr=flow_rule)
olca_model_A.demand_constraint.pprint()
olca_model_A.transport_constraint.pprint()
olca_model_A.flow_constraint.pprint()

demand_constraint : Size=1, Index=Fm, Active=True
    Key                                  : Lower  : Body                                       : Upper : Active
    9891d347-28f8-434d-896a-84a19a4c68a6 : 1000.0 : Flow[9891d347-28f8-434d-896a-84a19a4c68a6] :  +Inf :   True
transport_constraint : Size=3, Index=Ft, Active=True
    Key                                  : Lower : Body                                                                                                                                     : Upper : Active
    090f56b8-6a83-48c1-ae26-234d04771e53 :   0.0 : T[090f56b8-6a83-48c1-ae26-234d04771e53] - 1283.834155118865*f[9891d347-28f8-434d-896a-84a19a4c68a6,090f56b8-6a83-48c1-ae26-234d04771e53] :   0.0 :   True
    0ace02fa-eca5-482d-a829-c18e46a52db4 :   0.0 : T[0ace02fa-eca5-482d-a829-c18e46a52db4] - 1283.834155118865*f[9891d347-28f8-434d-896a-84a19a4c68a6,0ace02fa-eca5-482d-a829-c18e46a52db4] :   0.0 :   True
    2741cea8-327f-4e0f-9401-b10858dc68f8 :   0.0 : T[2741c

## Define objective

In [51]:
olca_model_A.obj = Objective(expr=sum(olca_model_A.Flow[f]*olca_model_A.EI_Pm[process[f]] for f in olca_model_A.Fm) +
                        sum(olca_model_A.T[f]*olca_model_A.EI_Pt[process[f]] for f in olca_model_A.Ft), sense=minimize)
olca_model_A.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 0.056119557120436626*T[0ace02fa-eca5-482d-a829-c18e46a52db4] + 1.84302999202496*T[090f56b8-6a83-48c1-ae26-234d04771e53] + 0.009366999860024065*T[2741cea8-327f-4e0f-9401-b10858dc68f8] + 0.13944742151786754*Flow[9891d347-28f8-434d-896a-84a19a4c68a6]


## Apply solver

In [52]:
opt = SolverFactory("glpk")
results = opt.solve(olca_model_A)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 12165.1217728104
  Upper bound: 12165.1217728104
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 8
  Number of nonzeros: 12
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.016128063201904297
# ----------------------------------------------------------
#   Solution Information
# -------------

## Generate results

In [53]:
olca_model_A.Flow.display()
olca_model_A.T.display()
olca_model_A.f.display()

Flow : Material flow
    Size=1, Index=Fm
    Key                                  : Lower : Value  : Upper : Fixed : Stale : Domain
    9891d347-28f8-434d-896a-84a19a4c68a6 :     0 : 1000.0 :  None : False : False : NonNegativeReals
T : Transport Service flow
    Size=3, Index=Ft
    Key                                  : Lower : Value            : Upper : Fixed : Stale : Domain
    090f56b8-6a83-48c1-ae26-234d04771e53 :     0 :              0.0 :  None : False : False : NonNegativeReals
    0ace02fa-eca5-482d-a829-c18e46a52db4 :     0 :              0.0 :  None : False : False : NonNegativeReals
    2741cea8-327f-4e0f-9401-b10858dc68f8 :     0 : 1283834.15511886 :  None : False : False : NonNegativeReals
f : Transport flow
    Size=3, Index=f_index
    Key                                                                              : Lower : Value  : Upper : Fixed : Stale : Domain
    ('9891d347-28f8-434d-896a-84a19a4c68a6', '090f56b8-6a83-48c1-ae26-234d04771e53') :     0 :    0.0 : 

In [54]:
 dv.get_process_product_flow(dbconn, ref_ids=material_ref_ids + transport_ref_ids)

SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_FLOWS"."NAME" "FLOW_NAME" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (5409534,10990570,25967156,33642886)) "e" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="e"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="e"."F_OWNER" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME
0,64867712-23c4-3be5-a50e-3631e74571a6,"lemon production | lemon | APOS, S",Spain,9891d347-28f8-434d-896a-84a19a4c68a6,lemon
1,b0d5d6ec-77f4-3213-b613-2391089d5f96,"transport, freight train, diesel | transport, ...",Europe without Switzerland,0ace02fa-eca5-482d-a829-c18e46a52db4,"transport, freight train"
2,3886c7a5-bb83-41d8-9b20-41c0df9742ce,"transport, freight, sea, container ship | tran...",Global,2741cea8-327f-4e0f-9401-b10858dc68f8,"transport, freight, sea, container ship"
3,13e79207-71e6-36c9-be76-716700ea1e81,"transport, freight, light commercial vehicle |...",Europe without Switzerland,090f56b8-6a83-48c1-ae26-234d04771e53,"transport, freight, light commercial vehicle"
